In [19]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split

In [27]:
class image_data_set(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, index):
        return {'data': self.data[index], 'label': self.labels[index]}

In [38]:
def get_image_tensor(file_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    image = Image.open(file_path)
    return transform(image)

def get_data_and_labels(directory_path, label):
    image_tensors, labels = [], []
    for file in os.listdir(directory_path):
        if file.endswith(".JPG"):
            file_path = directory_path + file
            image_tensor = get_image_tensor(file_path)
            image_tensors.append(image_tensor)
            labels.append(label)
            
    return image_tensors, labels
    
def get_data_sets(): 
    present_file_path = "data/MP152_ODOT009_REPELCAM/present/"
    not_present_file_path = "data/MP152_ODOT009_REPELCAM/not_present/"
    
    present_data, present_labels = get_data_and_labels(present_file_path, 1)
    not_present_data, not_present_labels = get_data_and_labels(not_present_file_path, 1)
    
    print("Number of wildlife present photos: " + str(len(present_data)))
    print("Number of no wildlife present photos: " + str(len(not_present_data)))
    
    data = present_data + not_present_data
    labels = present_labels + not_present_labels
    
    training_data, testing_data, training_labels, testing_labels = train_test_split(data, labels)
    
    print("\nNumber of training photos: " + str(len(training_data)))
    print("Number of testing photos: " + str(len(testing_data)))
    
    training_data_set = image_data_set(training_data, training_labels)
    testing_data_set = image_data_set(testing_data, testing_labels)
    
    return training_data_set, testing_data_set

def get_loaders(training_data_set, testing_data_set, batch_size):
    training_loader = torch.utils.data.DataLoader(dataset = training_data_set,
                                                  batch_size = batch_size,
                                                  shuffle = True)

    testing_loader = torch.utils.data.DataLoader(dataset = testing_data_set,
                                                 batch_size = batch_size,
                                                 shuffle = True)
    
    return training_loader, testing_loader

In [39]:
def train(model, training_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    num_correct = 0
    for i, data in enumerate(training_loader):
        data, labels = data['data'].to(device), data['label'].to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, labels)
        running_loss += loss.item()
        _, predictions = torch.max(output.data, 1)
        num_correct += (predictions == labels).sum().item()
        loss.backward()
        optimizer.step()
    
    loss = running_loss/len(training_loader.dataset)
    accuracy = num_correct/len(training_loader.dataset)
    return loss, accuracy

def test(model, testing_loader, criterion):
    model.eval()
    running_loss = 0.0
    num_correct = 0
    for i, data in enumerate(testing_loader):
        data, labels = data['data'].to(device), data['label'].to(device)
        output = model(data)
        loss = criterion(output, labels)
        running_loss += loss.item()
        _, predictions = torch.max(output.data, 1)
        num_correct += (predictions == labels).sum().item()
    
    loss = running_loss/len(testing_loader.dataset)
    accuracy = num_correct/len(testing_loader.dataset)
    return loss, accuracy

In [40]:
def train_and_test(model, training_loader, testing_loader, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(3):
        print("epoch: " + str(epoch))
        
        training_loss, training_accuracy = train(model, training_loader, criterion, optimizer)
        print("training loss: " + str(training_loss) + " and training accuracy: " + str(training_accuracy))
        
        testing_loss, testing_accuracy = test(model, testing_loader, criterion)
        print("testing loss: " + str(testing_loss) + " and testing accuracy: " + str(testing_accuracy))

In [41]:
def train_and_test_VGG(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing VGG")
    vgg11 = models.vgg11(weights = models.VGG11_Weights.DEFAULT)
    vgg11.classifier[6].out_features = num_classes
    train_and_test(vgg11, training_loader, testing_loader, device)

def train_and_test_ResNet(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing ResNet")
    resnet50 = models.resnet50(weights = models.ResNet50_Weights.DEFAULT)
    resnet50.fc.out_features = num_classes
    train_and_test(resnet50, training_loader, testing_loader, device)
    
def train_and_test_AlexNet(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing AlexNet")
    alexnet = models.alexnet(weights = models.AlexNet_Weights.IMAGENET1K_V1)
    alexnet.classifier[6].out_features = num_classes
    train_and_test(alexnet, training_loader, testing_loader, device)
    
def train_and_test_Faster_RCNN(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing Faster R-CNN")
    faster_rcnn = models.detection.fasterrcnn_resnet50_fpn_v2(weights = models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
    faster_rcnn.roi_heads.box_predictor.bbox_pred.out_features = num_classes
    train_and_test(faster_rcnn, training_loader, testing_loader, device)
    
def train_and_test_YOLO(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing YOLO")
    yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    train_and_test(yolo, training_loader, testing_loader, device)

In [43]:
num_classes = 2
batch_size = 10

training_data_set, testing_data_set = get_data_sets()
training_loader, testing_loader = get_loaders(training_data_set, testing_data_set, batch_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

train_and_test_VGG(training_loader, testing_loader, device, num_classes)
train_and_test_ResNet(training_loader, testing_loader, device, num_classes)
train_and_test_AlexNet(training_loader, testing_loader, device, num_classes)

#TODO: Get both of these working
#train_and_test_Faster_RCNN(training_loader, testing_loader, device, num_classes)
#train_and_test_YOLO(training_loader, testing_loader, device, num_classes)

Number of wildlife present photos: 223
Number of no wildlife present photos: 363

Number of training photos: 439
Number of testing photos: 147

Training and Testing VGG
epoch: 0
training loss: 0.03995401678730549 and training accuracy: 0.9544419134396356
testing loss: 0.0 and testing accuracy: 1.0
epoch: 1
training loss: 0.0 and training accuracy: 1.0
testing loss: 0.0 and testing accuracy: 1.0
epoch: 2
training loss: 0.0 and training accuracy: 1.0
testing loss: 0.0 and testing accuracy: 1.0

Training and Testing ResNet
epoch: 0
training loss: 0.23673341859180846 and training accuracy: 0.7107061503416856
testing loss: 0.011581134032077936 and testing accuracy: 1.0
epoch: 1
training loss: 0.0010477988715527542 and training accuracy: 1.0
testing loss: 0.0037402766661680477 and testing accuracy: 1.0
epoch: 2
training loss: 0.00012987285506633664 and training accuracy: 1.0
testing loss: 0.0027876274151803584 and testing accuracy: 1.0

Training and Testing AlexNet
epoch: 0
training loss: 0.